## 저장 프로시저 활용

#### (Oracle sqldeveloper)
oracle_data_new.sql 파일 실행 

#### (pycharm terminal)
```
python manage.py startapp procedure
```

#### config/settings.py 추가, (**순서중요)
```config/settings.py
INSTALLED_APPS = [
    ...
    'procedure',
]
```

#### procedure/models.py
```procedure/models.py
from django.db import models
from datetime import datetime

class Emp(models.Model):
    empno = models.IntegerField(primary_key=True)
    ename = models.CharField(max_length=50, null=False)
    job = models.CharField(max_length=50, null=False)
    hiredate = models.DateTimeField(default=datetime.now)
    sal = models.IntegerField(null=False, default = 0)

    def update_sal(self, sal):
        self.sal = sal
```

#### (pycharm terminal)
```
python manage.py makemigrations
python manage.py migrate
```

#### (sqldeveloper)
```
select * from tab;
select * from procedure_emp;
```

#### procedure/urls.py
```
from procedure import views
from django.urls import path

urlpatterns = [
    path('', views.home),
    # 사원테이블
    path('list_emp', views.list_emp),
    path('write_emp', views.write_emp),
    path('insert_emp', views.insert_emp),
    path('update_emp', views.update_emp),
    path('update_emp_p', views.update_emp_p),
    
    # 메모테이블 
    path('list_memo_p', views.list_memo_p),
    path('insert_memo_p', views.insert_memo_p),
    path('view_memo_p', views.view_memo_p),
    path('delete_memo_p', views.delete_memo_p),
    path('update_memo_p', views.update_memo_p),
    
    # CRUD : Create, Read, Update, Delete
]
```

#### config/urls.py
```config/urls.py
urlpatterns = [
    ...
    path('procedure/', include('procedure.urls')),
]
```

####
```
insert into procedure_emp values (7369,'이철수','사원','2010-12-17',300);
insert into procedure_emp values (7499,'이민수','대리','2011-02-20',360);
insert into procedure_emp values (7521,'박종수','대리','2012-02-22',425);
insert into procedure_emp values (7566,'임성민','팀장','2011-04-02',597);
insert into procedure_emp values (7654,'나호석','대리','2011-09-28',425);
insert into procedure_emp values (7698,'박성환','팀장','2021-05-01',585);
insert into procedure_emp values (7782,'손기철','팀장','2021-06-09',545);
insert into procedure_emp values (7788,'박기호','부장','2007-04-17',600);
insert into procedure_emp values (7839,'김철수','대표','2011-11-17',900);
insert into procedure_emp values (7844,'송명준','대리','2011-09-08',450);
insert into procedure_emp values (7876,'황선태','사원','2017-05-23',310);
insert into procedure_emp values (7900,'박민철','사원','2011-12-03',395);
insert into procedure_emp values (7902,'박희성','부장','2011-12-03',700);
insert into procedure_emp values (7934,'최철수','사원','2012-01-23',330);
commit;    # 반드시 커밋 할것
```

#### 저장 프로시저

function은 return값이 반드시 있음

procedure는 return값이 있을수도, 없을수도 있음

#### (sqldeveloper) 급여인상 프로시저
```
create or replace procedure mysal_p(p_empno number)
-- 생성    변경     프로시저    프로시저이름   (파라미터변수명 자료형(자료형을 꼭 써줘야 함))
is
-- 임시변수 / 선언부
begin
-- 실행부 
  update procedure_emp
  set sal=sal*1.1
  where empno=p_empno;  
end;
/
```
실행 후 에러가 없어야 함

```
insert into procedure_emp values (1,'김철수','대리',sysdate,500);
/** sysdate 현재시각 **/
select * from procedure_emp;  

execute mysal_p(1);  
-- 실행   프로시저 할당값 

select * from procedure_emp;

select * from user_source where name='MYSAL_P';
-- 프로시저의 값 실행되는 내용을 봄
-- user_source: 데이터사전, 시스템테이블 (저장 프로시저를 만들면 user_source라는 내장테이블에 기록이 됨)
-- name: 프로시저 이름, 소문자로 써서 만들더라도 따옴표 안에서는 반드시 대문자로 써야 함. (오라클에서는 내부적으로 대문자로 처리됨)
-- 'MYSAL_P'라는 프로시저를 찾아서 보여줌
-- sql parser sql 호출 시 해석 / sql optimizer 실행계획
-- 저장프로시저를 만들어두어 미리 빠르게 진행함
```



#### procedure/views.py
```procedure/views.py

from django.shortcuts import render, redirect
from procedure.models import Emp
import cx_Oracle

def home(request):
    return render(request, 'procedure/index.html')

def list_emp(request):
    empList = Emp.objects.order_by('ename')
    return render(request, 'procedure/list_emp.html',
                  {'empList': empList, 'empCount': len(empList)})

def update_emp(request):
    emp = Emp.objects.get(empno = request.GET['empno'])
    sal = int(request.GET['sal']) * 1.1
    emp_new = Emp(empno = emp.empno, ename = emp.ename,
                  job = emp.job, hiredate = emp.hiredate, sal = sal)
    emp_new.save()
    return redirect('/procedure/list_emp')

def update_emp_p(request):
    try:
        with cx_Oracle.connect('python/1234@localhost:1521/xe') as conn:
            with conn.cursor() as cursor:
                empno = request.GET['empno']
                cursor.callproc('maysal_p', [empno])
                conn.commit()
    except Exception as e:
        print(e)
    return redirect('/procedure/list_emp')

def write_emp(request):
    return render(request, 'procedure/write_emp.html')

def insert_emp(request):
    emp = Emp(empno=request.POST['empno'], ename = request.POST['ename'],
              job=request.POST['job'], hiredate=request.POST['hiredate'],
              sal=request.POST['sal'])
    emp.save()
    return redirect('/procedure/list_emp')

def list_memo_p(request):
    try:
        with cx_Oracle.connect('python/1234@localhost:1521/xe') as conn:
            with conn.cursor() as cursor:
                ref_cursor = conn.cursor()
                cursor.callproc('memo_list_p', [ref_cursor])
                rows = ref_cursor.fetchall()
    except Exception as e:
        print(e)
    return render(request, 'procedure/list_memo_p.html',
                      {'memoList': rows, 'cnt': len(rows)})

def insert_memo_p(request):
    try:
        with cx_Oracle.connect("python/1234@localhost:1521/xe") as conn:
            with conn.cursor() as cursor:
                writer = request.POST['writer']
                memo = request.POST['memo']
                cursor.callproc('memo_insert_p', [writer, memo])
                conn.commit()
    except Exception as e:
        print(e)
    return redirect('/procedure/list_memo_p')
def view_memo_p(request):
    try:
        with cx_Oracle.connect("python/1234@localhost:1521/xe") as conn:
            with conn.cursor() as cursor:
                idx = request.GET['idx']
                ref_cursor = conn.cursor()
                cursor.callproc('memo_view_p', [idx, ref_cursor])
                row = ref_cursor.fetchone()
    except Exception as e:
        print(e)
    return render(request, 'procedure/view_memo_p.html', {'memo': row})
def delete_memo_p(request):
    try:
        with cx_Oracle.connect("python/1234@localhost:1521/xe") as conn:
            with conn.cursor() as cursor:
                idx = request.GET['idx']
                cursor.callproc('memo_delete_p', [idx])
                conn.commit()
    except Exception as e:
        print(e)
    return redirect('/procedure/list_memo_p')
def update_memo_p(request):
    try:
        with cx_Oracle.connect("python/1234@localhost:1521/xe") as conn:
            with conn.cursor() as cursor:
                idx = request.POST['idx']
                writer = request.POST['writer']
                memo = request.POST['memo']
                cursor.callproc('memo_update_p', [idx, writer, memo])
                conn.commit()
    except Exception as e:
        print(e)
    return redirect('/procedure/list_memo_p')
```

#### address/templates/index.html
```address/templates/index.html
...
<a href="procedure/">저장프로시저 활용</a><br>
```

#### templates/procedure 디렉토리 생성

#### procedure/templates/procedure/index.html
```procedure/templates/procedure/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
<a href = "/procedure/list_emp">사원명부</a><br>
<a href = "/procedure/list_memo_p">메모장</a><br>
<a href = "/">Home</a><br>
</body>
</html>
```

#### (pycharm terminal) 웹서버구동
```
python manage.py runserver
```

#### procedure/templates/procedure/list_emp.html
```procedure/templates/procedure/list_emp.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  <script>
    function update_sal(empno,sal){
      location.href="/procedure/update_emp?empno="+empno+"&sal="+sal;
    }
    function update_sal_p(empno){
      location.href="/procedure/update_emp_p?empno="+empno;
    }
  </script>
</head>
<body>
<h2>사원</h2>
<a href="/procedure/write_emp">사원 추가</a>
<a href="/procedure">프로시저 메인</a>
<a href="/">Home</a>
<table border="1">
  <tr>
    <th>사번</th>
    <th>이름</th>
    <th>직급</th>
    <th>입사일자</th>
    <th>급여</th>
    <th>&nbsp;</th>
  </tr>
  {% for emp in empList %}
  <tr>
    <td>{{emp.empno}}</td>
    <td>{{emp.ename}}</td>
    <td>{{emp.job}}</td>
    <td>{{emp.hiredate|date:'Y-m-d'}}</td>
    <td>{{emp.sal}}</td>
    <td>
      <input type="button" value="급여인상" onclick="update_sal('{{emp.empno}}','{{emp.sal}}')">
      <input type="button" value="급여인상(프로시저)" onclick="update_sal_p('{{emp.empno}}')">
    </td>
  </tr>
  {% endfor %}
</table>
</body>
</html>

```

### procedure/templates/procedure/write_emp.html
```procedure/templates/procedure/write_emp.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
<h2>사원 정보</h2>
<form method="post" action="insert_emp">
  {% csrf_token %}
  <table border="1">
    <tr>
      <td>사번</td>
      <td><input name="empno"></td>
    </tr>
    <tr>
      <td>이름</td>
      <td><input name="ename"></td>
    </tr>
    <tr>
      <td>직급</td>
      <td><input name="job"></td>
    </tr>
    <tr>
      <td>입사일자</td>
      <td><input type="date" name="hiredate"></td>
    </tr>
    <tr>
      <td>급여</td>
      <td><input type="number" name="sal"></td>
    </tr>
    <tr align="center">
      <td colspan="2">
        <input type="submit" value="확인">
      </td>
    </tr>
  </table>
</form>
</body>
</html>

```
